In [8]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import re
import firebase_admin
from firebase_admin import credentials, firestore

# 1. 이메일 읽기
with open('received_email.txt', 'r') as file:
    email = file.read().strip()
    print(f"이메일: {email}")

# 2. Firebase Admin SDK 초기화
if not len(firebase_admin._apps):
    cred = credentials.Certificate('C:/pododoc/key/kosmo-96bbe-60906db745e9.json')
    firebase_admin.initialize_app(cred)

# 3. Firestore 클라이언트 생성
db = firestore.client()

# 4. 이메일로 데이터 조회하는 함수 정의
def fetch_data_by_email(email):
    reviews_ref = db.collection('review')
    query = reviews_ref.where('email', '==', email)
    results = query.stream()
    data = []
    for doc in results:
        doc_data = doc.to_dict()
        if 'index' in doc_data and 'rating' in doc_data and 'date' in doc_data and 'photo' in doc_data:
            data.append({
                'index': doc_data['index'],
                'rating': doc_data['rating'],
            })
    return data

# 5. 이메일에 맞는 데이터 가져오기
data = fetch_data_by_email(email)

# 6. 결과 출력
print("가져온 데이터:")
for entry in data:
    print(f"Index: {entry['index']}, Rating: {entry['rating']}")
    
# 7. Combined_Wine_Data.csv 파일 읽기
wine_data = pd.read_csv('data/Combined_Wine_Data.csv')

# 8. CSV 파일의 컬럼 이름 확인하기
print("CSV 파일의 컬럼 이름:")
print(wine_data.columns)

# 9. index 값에 해당하는 와인 데이터 찾기
indices = [entry['index'] for entry in data]
filtered_wine_data = wine_data[wine_data['index'].isin(indices)]
filtered_wine_data = filtered_wine_data[['index', 'wine_country', 'wine_grape', 'wine_name', 'wine_region', 'wine_winery']]










# 와인 유사도 찾기 함수
def find_similar_wines_from_name(target_index):
    # 데이터 로딩
    df = pd.read_csv('data/Clean_Red_data.csv')

    def find_wine_name(target_index):
        return df.loc[df['index'] == target_index, 'wine_name'].values[0]
    
    target_wine_name = find_wine_name(target_index)
    
    # 평점 가중치 계산
    m = df['wine_reviews'].min()
    C = df['wine_rating'].mean()
    
    def weighted_rating(x, m=m, C=C):
        v = x['wine_reviews']
        R = x['wine_rating']
        return (v/(v+m) * R) + (m/(v+m) * C)
    
    df['point'] = df.apply(weighted_rating, axis=1)

    # 정규화 (0에서 1 사이로 변환)
    df_features = df[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    for column in ['body', 'texture', 'sweetness', 'acidity']:
        df_features[column] = df_features[column] / 100.0

    # ColumnTransformer 정의: 원-핫 인코딩 및 표준화 적용
    ct = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(drop='first'), ['flavor1', 'flavor2', 'flavor3']),  # 범주형 데이터 원-핫 인코딩
            ('scaler', StandardScaler(), ['body', 'texture', 'sweetness', 'acidity'])  # 숫자형 데이터 표준화
        ],
        remainder='passthrough'
    )

    # 변환 적용
    X = ct.fit_transform(df_features)

    def calculate_distance_with_flavors(df, target_index, ct, columns):
        # 모든 feature를 인코딩 및 스케일링된 형태로 변환
        df_encoded = ct.transform(df[columns])
        
        # 특정 와인 벡터
        target_vector = df_encoded[target_index, :].reshape(1, -1)
        
        # 데이터 벡터
        data_vectors = df_encoded
        
        # 유클리디안 거리 계산
        distance_scores = euclidean_distances(target_vector, data_vectors)
        
        # 결과 반환
        return distance_scores.flatten()

    def find_wine_index(df, wine_name):
        return df[df['wine_name'] == wine_name].index[0]

    def remove_year_from_name(wine_name):
        """ 와인 이름에서 연도나 숫자 부분을 제거합니다. """
        return re.sub(r'\d+', '', wine_name).strip()

    def filter_unique_wine_names(df):
        """ 와인 이름에서 숫자 부분을 제거하고, 고유한 와인 이름만 필터링합니다. """
        # 와인 이름에서 숫자 부분을 제거한 새로운 컬럼 생성
        df['wine_name_clean'] = df['wine_name'].apply(remove_year_from_name)
        
        # 중복 와인 이름 제거 (최소 거리 와인만 남기기)
        df_sorted_filtered = df.sort_values(by='distance', ascending=True)
        df_unique = df_sorted_filtered.drop_duplicates(subset='wine_name_clean', keep='first')
        
        # 불필요한 컬럼 삭제
        df_unique = df_unique.drop(columns=['wine_name_clean'])
        
        return df_unique

    columns = ['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']
    target_index = find_wine_index(df, target_wine_name)
    distances = calculate_distance_with_flavors(df, target_index, ct, columns)
    
    # 유사도와 데이터프레임 결합 (유클리디안 거리이므로 거리 값이 작은 순서가 유사도 높은 것)
    df['distance'] = distances
    
    # 거리 값이 작은 순으로 정렬
    df_sorted = df.sort_values(by='distance', ascending=True)
    
    # `point`가 4점 이상인 와인만 필터링
    df_sorted_filtered = df_sorted[df_sorted['point'] >= 2.5]

    # 중복된 와인 이름 제거
    df_unique = filter_unique_wine_names(df_sorted_filtered)
    
    return df_unique

# 테스트 실행
target_index = 152  # 예시 인덱스, 실제로는 데이터에 맞는 인덱스를 입력해야 합니다.
df_sorted = find_similar_wines_from_name(target_index)

# 상위 10개 와인 출력
import IPython.display as display
display.display(df_sorted.head(10))


이메일: blue@test.com
가져온 데이터:
Index: 5, Rating: 2.5
Index: 734, Rating: 2.0
Index: 180, Rating: 3.5
Index: 1969, Rating: 5.0
Index: 249, Rating: 5.0
Index: 5, Rating: 2.5999999046325684
Index: 5, Rating: 4.900000095367432
Index: 1982, Rating: 3.0999999046325684
Index: 973, Rating: 2.0
Index: 5, Rating: 4.0
Index: 1972, Rating: 2.5
Index: 5, Rating: 2.5
Index: 1668, Rating: 1.5
Index: 486, Rating: 4.0
Index: 1975, Rating: 2.200000047683716
Index: 1978, Rating: 1.7999999523162842


,index,wine_name,wine_rating,wine_reviews,wine_price,wine_link,wine_country,wine_region,wine_winery,wine_type,...,wine_image,body,texture,sweetness,acidity,flavor1,flavor2,flavor3,point,distance
148,152,Pauillac 1970,4.0,6482,"148,392",https://www.vivino.com/chateau-pibran-pauillac...,France,Pauillac,Château Pibran,Red wine,...,https://images.vivino.com/thumbs/iV5kWnk4QZ-hp...,82.2426,81.2348,13.48051,81.1337,oaky,black fruit,earthy,4.000286,0.000000
1028,1110,Pauillac (Grand Cru Classé) 1995,4.0,310,"105,106",https://www.vivino.com/fr-chateau-haut-bages-l...,France,Pauillac,Château Haut-Bages Libéral,Red wine,...,https://images.vivino.com/thumbs/JOiOnT_uRpOGs...,80.4582,81.3111,13.17549,80.3218,oaky,black fruit,earthy,4.005002,13.677234
58,60,Pessac-Léognan 1966,4.0,16162,"222,588",https://www.vivino.com/chateau-la-louviere-pes...,France,Pessac-Léognan,Château La Louvière,Red wine,...,https://images.vivino.com/thumbs/s8MbGOeBTXm2X...,80.1777,81.7625,13.83615,80.3232,oaky,earthy,black fruit,4.000115,15.955180
650,699,Saint-Estèphe 2014,4.0,721,"68,008",https://www.vivino.com/ormes-de-pez-saint-este...,France,Saint-Estèphe,Château Ormes de Pez,Red wine,...,https://images.vivino.com/thumbs/xxPubHfBQV6-T...,82.8207,83.0949,14.41074,80.6502,black fruit,oaky,earthy,4.002383,18.577569
1255,1358,Le Clarence de Haut Brion Pessac-Léognan (Chât...,4.1,196,"179,302",https://www.vivino.com/haut-brion-le-clarence-...,France,Pessac-Léognan,Château Haut-Brion,Red wine,...,https://images.vivino.com/thumbs/YSmFivppSEaff...,79.5985,80.2772,13.59021,80.1606,oaky,black fruit,earthy,4.082579,19.627081
1766,1929,Haut-Médoc (Grand Cru Classé) 1986,4.2,69,"105,106",https://www.vivino.com/chateau-belgrave-haut-m...,France,Haut-Médoc,Château Belgrave,Red wine,...,https://images.vivino.com/thumbs/0637pHsxRoWpA...,79.6327,79.9965,13.56574,79.1563,oaky,black fruit,earthy,4.117825,23.174293
1606,1754,Haut-Médoc 2003,3.8,94,"45,836",https://www.vivino.com/fr-chateau-beaumont-hau...,France,Haut-Médoc,Château Beaumont,Red wine,...,https://images.vivino.com/thumbs/mvke5H4aQVChl...,80.8114,79.3979,12.17548,80.3574,black fruit,oaky,earthy,3.892852,24.581915
1400,1521,Les Tourelles de Longueville Pauillac 1996,4.3,145,"111,294",https://www.vivino.com/chateau-pichon-baron-le...,France,Pauillac,Château Pichon Baron,Red wine,...,https://images.vivino.com/thumbs/_b9CdfKRRQOBD...,81.0125,77.9079,13.14062,79.3915,oaky,black fruit,earthy,4.217084,26.035962
1685,1840,La Dame de Montrose Saint-Estèphe 1990,4.6,80,"283,176",https://www.vivino.com/chateau-montrose-la-dam...,France,Saint-Estèphe,Château Montrose,Red wine,...,https://images.vivino.com/thumbs/OuCPyNv9QQier...,81.5943,82.2834,12.84694,77.5683,black fruit,oaky,earthy,4.346324,26.990022
925,994,Lacoste Borie Pauillac 2006,4.0,391,"66,776",https://www.vivino.com/chateau-grand-puy-lacos...,France,Pauillac,Château Grand-Puy-Lacoste,Red wine,...,https://images.vivino.com/thumbs/fvJe1w6CRuWph...,79.7458,78.3357,13.59079,79.4034,oaky,black fruit,earthy,4.004111,27.396261


In [36]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
from collections import Counter

# 1. 이메일 읽기
with open('received_email.txt', 'r') as file:
    email = file.read().strip()
    print(f"이메일: {email}")

# 2. Firebase Admin SDK 초기화
if not len(firebase_admin._apps):
    cred = credentials.Certificate('C:/pododoc/key/kosmo-96bbe-60906db745e9.json')
    firebase_admin.initialize_app(cred)

# 3. Firestore 클라이언트 생성
db = firestore.client()

# 4. 이메일로 데이터 조회하는 함수 정의
def fetch_data_by_email(email):
    reviews_ref = db.collection('review')
    query = reviews_ref.where('email', '==', email)
    results = query.stream()
    data = []
    for doc in results:
        doc_data = doc.to_dict()
        if 'index' in doc_data and 'rating' in doc_data and 'date' in doc_data and 'photo' in doc_data:
            data.append({
                'index': doc_data['index'],
                'rating': doc_data['rating'],
            })
    return data

# 5. 이메일에 맞는 데이터 가져오기
data = fetch_data_by_email(email)

# 6. 결과 출력
print("가져온 데이터:")
for entry in data:
    print(f"Index: {entry['index']}, Rating: {entry['rating']}")

# 7. Combined_Wine_Data.csv 파일 읽기
wine_data = pd.read_csv('data/Clean_Red_data.csv')

# 9. index 값에 해당하는 와인 데이터 찾기
indices = [entry['index'] for entry in data]
filtered_wine_data = wine_data[wine_data['index'].isin(indices)]

# 10. 추가적인 컬럼을 포함하여 필요한 컬럼 선택하기
required_columns = ['index', 'body', 'texture', 'sweetness', 'acidity', 'flavor1', 'flavor2', 'flavor3']
filtered_wine_data = filtered_wine_data[required_columns]

# 11. 이메일로 가져온 데이터에서 rating도 포함시키기 위해, rating 정보를 추가
# rating 정보를 dictionary 형태로 변환
rating_dict = {entry['index']: entry['rating'] for entry in data}

# rating 정보를 filtered_wine_data에 추가
filtered_wine_data['rating'] = filtered_wine_data['index'].map(rating_dict)

# 12. 데이터 타입 확인 및 변환
# 변환할 컬럼 리스트
numeric_columns = ['body', 'texture', 'sweetness', 'acidity']

# 숫자형 컬럼의 데이터 변환 (문자열이 포함된 경우 NaN으로 변환)
filtered_wine_data[numeric_columns] = filtered_wine_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# NaN 값이 있는 행 제거
filtered_wine_data = filtered_wine_data.dropna()

# 13. 가중 평균 계산 함수 정의
def weighted_average(column):
    weights = filtered_wine_data['rating']
    values = filtered_wine_data[column]
    return (weights * values).sum() / weights.sum()

# 각 숫자형 컬럼에 대해 가중 평균 계산
weighted_averages = {
    'body': weighted_average('body'),
    'texture': weighted_average('texture'),
    'sweetness': weighted_average('sweetness'),
    'acidity': weighted_average('acidity')
}

# 14. flavor1, flavor2, flavor3에서 가장 많이 등장한 항목 찾기
def weighted_mode(column):
    counts = Counter(filtered_wine_data[column])
    weighted_counts = {}
    for item in counts:
        weighted_counts[item] = (filtered_wine_data[filtered_wine_data[column] == item]['rating']).sum()
    return max(weighted_counts, key=weighted_counts.get)

flavor1_mode = weighted_mode('flavor1')
flavor2_mode = weighted_mode('flavor2')
flavor3_mode = weighted_mode('flavor3')

# 15. 결과를 데이터프레임으로 저장
result_df = pd.DataFrame({
    'flavor1': [flavor1_mode],
    'flavor2': [flavor2_mode],
    'flavor3': [flavor3_mode],
    'body': [weighted_averages['body']],
    'texture': [weighted_averages['texture']],
    'sweetness': [weighted_averages['sweetness']],
    'acidity': [weighted_averages['acidity']]
})

# 16. 결과 출력
print("\n최종 결과:")
print(result_df)


이메일: blue@test.com
가져온 데이터:
Index: 5, Rating: 2.5
Index: 734, Rating: 2.0
Index: 180, Rating: 3.5
Index: 1969, Rating: 5.0
Index: 249, Rating: 5.0
Index: 5, Rating: 2.5999999046325684
Index: 5, Rating: 4.900000095367432
Index: 1982, Rating: 3.0999999046325684
Index: 973, Rating: 2.0
Index: 5, Rating: 4.0
Index: 1972, Rating: 2.5
Index: 5, Rating: 2.5
Index: 1668, Rating: 1.5
Index: 486, Rating: 4.0
Index: 1975, Rating: 2.200000047683716
Index: 1978, Rating: 1.7999999523162842

최종 결과:
  flavor1      flavor2    flavor3       body    texture  sweetness  acidity
0    oaky  black fruit  red fruit  66.972678  51.596661  22.490718  59.4616


c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [72]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import re

# 1. `result_df`와 유사한 와인 찾기
def find_similar_wines_from_result_df(target_index, original_data):
    # 데이터 준비
    df_features = result_df[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    original_features = original_data[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    
    # ColumnTransformer 정의: 원-핫 인코딩 및 표준화 적용
    ct = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'), ['flavor1', 'flavor2', 'flavor3']),
            ('scaler', StandardScaler(), ['body', 'texture', 'sweetness', 'acidity'])
        ],
        remainder='passthrough'
    )

    # 변환 적용
    X = ct.fit_transform(df_features)
    X_original = ct.transform(original_features)

    def calculate_distance_with_flavors(df_encoded, target_index, original_encoded):
        # 특정 와인 벡터
        target_vector = df_encoded[target_index, :].reshape(1, -1)
        
        # 유클리디안 거리 계산
        distance_scores = euclidean_distances(target_vector, original_encoded)
        
        # 결과 반환
        return distance_scores.flatten()

    # 거리 계산
    distances = calculate_distance_with_flavors(X, target_index, X_original)
    
    # 자기 자신과의 거리 제외
    distances[target_index] = float('inf')

    # 유사한 와인 찾기
    similar_indices = distances.argsort()  # 모든 와인 정렬
    
    # 상위 10개 와인 선택
    top_similar_indices = similar_indices[:20]  # 더 많은 와인을 선택하여 중복을 제거
    
    # 결과에서 index 값을 추출
    similar_wines = original_data.iloc[top_similar_indices].copy()
    similar_wines['distance'] = distances[top_similar_indices]
    
    # 거리 순서로 정렬
    similar_wines = similar_wines.sort_values(by='distance')
    
    return similar_wines[['index', 'distance']]

# 2. `data/Clean_Red_data.csv`에서 원본 데이터 읽기 및 인덱스 값 확인
def load_and_prepare_data(csv_file_path):
    # CSV 파일 읽기
    original_data = pd.read_csv(csv_file_path)
    
    # `index` 컬럼이 있는지 확인하고, 없으면 생성
    if 'index' not in original_data.columns:
        original_data = original_data.reset_index()
        original_data.rename(columns={'index': 'index'}, inplace=True)
    
    return original_data

# 3. 유사한 와인의 추가 정보 추출 및 중복 제거
def get_wine_details(similar_wines_df, original_data):
    # 유사한 와인의 인덱스 리스트 추출
    similar_indices = similar_wines_df['index'].tolist()
    
    # 원본 데이터에서 해당 인덱스에 해당하는 와인 정보 추출
    wine_details = original_data[original_data['index'].isin(similar_indices)]
    
    # 유사한 와인 순서에 맞춰 정렬
    wine_details = wine_details.set_index('index').reindex(similar_indices).reset_index()
    
    # 거리 정보 추가
    wine_details = wine_details.merge(similar_wines_df, on='index')
    
    # 필요한 컬럼만 선택
    columns_of_interest = [
        'index', 'wine_name', 'wine_rating', 'wine_price',
        'wine_country', 'wine_region', 'wine_winery', 'wine_type', 'wine_grape', 'wine_image', 'distance'
    ]
    
    # 정보 추출
    wine_details = wine_details[columns_of_interest]
    
    # 와인 이름의 숫자 이전 부분만 추출하여 중복 제거
    def extract_name_prefix(name):
        match = re.match(r'^(.*\D)', name)
        return match.group(1) if match else name
    
    wine_details['name_prefix'] = wine_details['wine_name'].apply(extract_name_prefix)
    
    # 이름의 접두사로 중복 제거
    wine_details = wine_details.drop_duplicates(subset='name_prefix')
    
    # 원래의 최대 10개 와인으로 제한
    if len(wine_details) > 10:
        wine_details = wine_details.head(10)
    
    # 거리로 정렬
    wine_details = wine_details.sort_values(by='distance')
    
    # 불필요한 컬럼 삭제
    wine_details = wine_details.drop(columns='name_prefix')
    
    return wine_details

# `result_df`와 `data/Clean_Red_data.csv`에서 가장 유사한 와인 찾기
csv_file_path = 'data/Clean_Red_data.csv'
original_data = load_and_prepare_data(csv_file_path)

# `result_df`가 하나의 와인만 포함한다고 가정
target_index = 0  # `result_df`의 첫 번째 와인

# 가장 유사한 와인 찾기
similar_wines_indices = find_similar_wines_from_result_df(target_index, original_data)

# 유사한 와인의 상세 정보 추출
wine_details = get_wine_details(similar_wines_indices, original_data)

# 결과를 CSV 파일로 저장
output_csv_path = 'similar_wines_details.csv'
wine_details.to_csv(output_csv_path, index=False)

print(f"가장 유사한 와인의 상세 정보가 '{output_csv_path}' 파일로 저장되었습니다.")


가장 유사한 와인의 상세 정보가 'similar_wines_details.csv' 파일로 저장되었습니다.


c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [77]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
from collections import Counter

# 1. 이메일 읽기
with open('received_email.txt', 'r') as file:
    email = file.read().strip()
    print(f"이메일: {email}")

# 2. Firebase Admin SDK 초기화
if not len(firebase_admin._apps):
    cred = credentials.Certificate('C:/pododoc/key/kosmo-96bbe-60906db745e9.json')
    firebase_admin.initialize_app(cred)

# 3. Firestore 클라이언트 생성
db = firestore.client()

# 4. 이메일로 데이터 조회하는 함수 정의
def fetch_data_by_email(email):
    reviews_ref = db.collection('review')
    query = reviews_ref.where('email', '==', email)
    results = query.stream()
    data = []
    for doc in results:
        doc_data = doc.to_dict()
        if 'index' in doc_data and 'rating' in doc_data and 'date' in doc_data and 'photo' in doc_data:
            data.append({
                'index': doc_data['index'],
                'rating': doc_data['rating'],
            })
    return data

# 5. 이메일에 맞는 데이터 가져오기
data = fetch_data_by_email(email)

# 6. 결과 출력
print("가져온 데이터:")
for entry in data:
    print(f"Index: {entry['index']}, Rating: {entry['rating']}")

# 7. Combined_Wine_Data.csv 파일 읽기
wine_data = pd.read_csv('data/Clean_Red_data.csv')

# 9. index 값에 해당하는 와인 데이터 찾기
indices = [entry['index'] for entry in data]
filtered_wine_data = wine_data[wine_data['index'].isin(indices)]

# 10. 추가적인 컬럼을 포함하여 필요한 컬럼 선택하기
required_columns = ['index', 'body', 'texture', 'sweetness', 'acidity', 'flavor1', 'flavor2', 'flavor3']
filtered_wine_data = filtered_wine_data[required_columns]

# 11. 이메일로 가져온 데이터에서 rating도 포함시키기 위해, rating 정보를 추가
# rating 정보를 dictionary 형태로 변환
rating_dict = {entry['index']: entry['rating'] for entry in data}

# rating 정보를 filtered_wine_data에 추가
filtered_wine_data['rating'] = filtered_wine_data['index'].map(rating_dict)

# 12. 데이터 타입 확인 및 변환
# 변환할 컬럼 리스트
numeric_columns = ['body', 'texture', 'sweetness', 'acidity']

# 숫자형 컬럼의 데이터 변환 (문자열이 포함된 경우 NaN으로 변환)
filtered_wine_data[numeric_columns] = filtered_wine_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# NaN 값이 있는 행 제거
filtered_wine_data = filtered_wine_data.dropna()

# 13. 가중 평균 계산 함수 정의
def weighted_average(column):
    weights = filtered_wine_data['rating']
    values = filtered_wine_data[column]
    return (weights * values).sum() / weights.sum()

# 각 숫자형 컬럼에 대해 가중 평균 계산
weighted_averages = {
    'body': weighted_average('body'),
    'texture': weighted_average('texture'),
    'sweetness': weighted_average('sweetness'),
    'acidity': weighted_average('acidity')
}

# 14. flavor1, flavor2, flavor3에서 가장 많이 등장한 항목 찾기
def weighted_mode(column):
    counts = Counter(filtered_wine_data[column])
    weighted_counts = {}
    for item in counts:
        weighted_counts[item] = (filtered_wine_data[filtered_wine_data[column] == item]['rating']).sum()
    return max(weighted_counts, key=weighted_counts.get)

flavor1_mode = weighted_mode('flavor1')
flavor2_mode = weighted_mode('flavor2')
flavor3_mode = weighted_mode('flavor3')

# 15. 결과를 데이터프레임으로 저장
result_df = pd.DataFrame({
    'flavor1': [flavor1_mode],
    'flavor2': [flavor2_mode],
    'flavor3': [flavor3_mode],
    'body': [weighted_averages['body']],
    'texture': [weighted_averages['texture']],
    'sweetness': [weighted_averages['sweetness']],
    'acidity': [weighted_averages['acidity']]
})

# 16. 결과 출력
print("\n최종 결과:")
print(result_df)

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import re

# 1. `result_df`와 유사한 와인 찾기
def find_similar_wines_from_result_df(target_index, original_data):
    # 데이터 준비
    df_features = result_df[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    original_features = original_data[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    
    # ColumnTransformer 정의: 원-핫 인코딩 및 표준화 적용
    ct = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'), ['flavor1', 'flavor2', 'flavor3']),
            ('scaler', StandardScaler(), ['body', 'texture', 'sweetness', 'acidity'])
        ],
        remainder='passthrough'
    )

    # 변환 적용
    X = ct.fit_transform(df_features)
    X_original = ct.transform(original_features)

    def calculate_distance_with_flavors(df_encoded, target_index, original_encoded):
        # 특정 와인 벡터
        target_vector = df_encoded[target_index, :].reshape(1, -1)
        
        # 유클리디안 거리 계산
        distance_scores = euclidean_distances(target_vector, original_encoded)
        
        # 결과 반환
        return distance_scores.flatten()

    # 거리 계산
    distances = calculate_distance_with_flavors(X, target_index, X_original)
    
    # 자기 자신과의 거리 제외
    distances[target_index] = float('inf')

    # 유사한 와인 찾기
    similar_indices = distances.argsort()  # 모든 와인 정렬
    
    # 상위 10개 와인 선택
    top_similar_indices = similar_indices[:20]  # 더 많은 와인을 선택하여 중복을 제거
    
    # 결과에서 index 값을 추출
    similar_wines = original_data.iloc[top_similar_indices].copy()
    similar_wines['distance'] = distances[top_similar_indices]
    
    # 거리 순서로 정렬
    similar_wines = similar_wines.sort_values(by='distance')
    
    return similar_wines[['index', 'distance']]

# 2. `data/Clean_Red_data.csv`에서 원본 데이터 읽기 및 인덱스 값 확인
def load_and_prepare_data(csv_file_path):
    # CSV 파일 읽기
    original_data = pd.read_csv(csv_file_path)
    
    # `index` 컬럼이 있는지 확인하고, 없으면 생성
    if 'index' not in original_data.columns:
        original_data = original_data.reset_index()
        original_data.rename(columns={'index': 'index'}, inplace=True)
    
    return original_data

# 3. 유사한 와인의 추가 정보 추출 및 중복 제거
def get_wine_details(similar_wines_df, original_data):
    # 유사한 와인의 인덱스 리스트 추출
    similar_indices = similar_wines_df['index'].tolist()
    
    # 원본 데이터에서 해당 인덱스에 해당하는 와인 정보 추출
    wine_details = original_data[original_data['index'].isin(similar_indices)]
    
    # 유사한 와인 순서에 맞춰 정렬
    wine_details = wine_details.set_index('index').reindex(similar_indices).reset_index()
    
    # 거리 정보 추가
    wine_details = wine_details.merge(similar_wines_df, on='index')
    
    # 필요한 컬럼만 선택
    columns_of_interest = [
        'index', 'wine_name', 'wine_rating', 'wine_price',
        'wine_country', 'wine_region', 'wine_winery', 'wine_type', 'wine_grape', 'wine_image', 'distance'
    ]
    
    # 정보 추출
    wine_details = wine_details[columns_of_interest]
    
    # 와인 이름의 숫자 이전 부분만 추출하여 중복 제거
    def extract_name_prefix(name):
        match = re.match(r'^(.*\D)', name)
        return match.group(1) if match else name
    
    wine_details['name_prefix'] = wine_details['wine_name'].apply(extract_name_prefix)
    
    # 이름의 접두사로 중복 제거
    wine_details = wine_details.drop_duplicates(subset='name_prefix')
    
    # 원래의 최대 10개 와인으로 제한
    if len(wine_details) > 10:
        wine_details = wine_details.head(10)
    
    # 거리로 정렬
    wine_details = wine_details.sort_values(by='distance')
    
    # 불필요한 컬럼 삭제
    wine_details = wine_details.drop(columns='name_prefix')
    
    return wine_details

# `result_df`와 `data/Clean_Red_data.csv`에서 가장 유사한 와인 찾기
csv_file_path = 'data/Clean_Red_data.csv'
original_data = load_and_prepare_data(csv_file_path)

# `result_df`가 하나의 와인만 포함한다고 가정
target_index = 0  # `result_df`의 첫 번째 와인

# 가장 유사한 와인 찾기
similar_wines_indices = find_similar_wines_from_result_df(target_index, original_data)

# 유사한 와인의 상세 정보 추출
wine_details = get_wine_details(similar_wines_indices, original_data)

print(wine_details)


이메일: blue@test.com
가져온 데이터:
Index: 5, Rating: 2.5
Index: 734, Rating: 2.0
Index: 180, Rating: 3.5
Index: 1969, Rating: 5.0
Index: 249, Rating: 5.0
Index: 5, Rating: 2.5999999046325684
Index: 5, Rating: 4.900000095367432
Index: 1982, Rating: 3.0999999046325684
Index: 973, Rating: 2.0
Index: 5, Rating: 4.0
Index: 1972, Rating: 2.5
Index: 5, Rating: 2.5
Index: 1668, Rating: 1.5
Index: 486, Rating: 4.0
Index: 1975, Rating: 2.200000047683716
Index: 1978, Rating: 1.7999999523162842

최종 결과:
  flavor1      flavor2    flavor3       body    texture  sweetness  acidity
0    oaky  black fruit  red fruit  66.972678  51.596661  22.490718  59.4616
    index                                     wine_name  wine_rating  \
0    1028                   Réserve Côtes-du-Rhône 2008          4.4   
5      51                          Original Merlot 2004          3.5   
6    1168  Plantier de La Reine Bordeaux Superieur 2004          3.7   
7     674                                 L'Heravi 2010          3.5   

c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [101]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import re

# 가격 범위에 따른 필터링 함수 정의
def filter_by_price(df_red_wine, price_range):
    # 'wine_price' 컬럼이 문자열인 경우 숫자로 변환
    df_red_wine['wine_price'] = df_red_wine['wine_price'].replace('[\$,]', '', regex=True).astype(float)
    
    # 가격 범위에 따른 필터링
    if price_range == '50000':
        df_filtered = df_red_wine[df_red_wine['wine_price'] <= 50000]
    elif price_range == '150000':
        df_filtered = df_red_wine[(df_red_wine['wine_price'] > 50000) & (df_red_wine['wine_price'] <= 150000)]
    elif price_range == 'over150000':
        df_filtered = df_red_wine[df_red_wine['wine_price'] > 150000]
    else:
        df_filtered = df_red_wine
    
    return df_filtered

# 1. `result_df`와 유사한 와인 찾기
def find_similar_wines_from_result_df(target_index, original_data):
    # 데이터 준비
    df_features = result_df[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    original_features = original_data[['flavor1', 'flavor2', 'flavor3', 'body', 'texture', 'sweetness', 'acidity']].copy()
    
    # ColumnTransformer 정의: 원-핫 인코딩 및 표준화 적용
    ct = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'), ['flavor1', 'flavor2', 'flavor3']),
            ('scaler', StandardScaler(), ['body', 'texture', 'sweetness', 'acidity'])
        ],
        remainder='passthrough'
    )

    # 변환 적용
    X = ct.fit_transform(df_features)
    X_original = ct.transform(original_features)

    def calculate_distance_with_flavors(df_encoded, target_index, original_encoded):
        # 특정 와인 벡터
        target_vector = df_encoded[target_index, :].reshape(1, -1)
        
        # 유클리디안 거리 계산
        distance_scores = euclidean_distances(target_vector, original_encoded)
        
        # 결과 반환
        return distance_scores.flatten()

    # 거리 계산
    distances = calculate_distance_with_flavors(X, target_index, X_original)
    
    # 자기 자신과의 거리 제외
    distances[target_index] = float('inf')

    # 유사한 와인 찾기
    similar_indices = distances.argsort()  # 모든 와인 정렬
    
    # 상위 20개 와인 선택
    top_similar_indices = similar_indices[:20]  # 더 많은 와인을 선택하여 중복을 제거
    
    # 결과에서 index 값을 추출
    similar_wines = original_data.iloc[top_similar_indices].copy()
    similar_wines['distance'] = distances[top_similar_indices]
    
    # 거리 순서로 정렬
    similar_wines = similar_wines.sort_values(by='distance')
    
    return similar_wines[['index', 'distance']]

# 2. `data/Clean_Red_data.csv`에서 원본 데이터 읽기 및 인덱스 값 확인
def load_and_prepare_data(csv_file_path):
    # CSV 파일 읽기
    original_data = pd.read_csv(csv_file_path)
    
    # `index` 컬럼이 있는지 확인하고, 없으면 생성
    if 'index' not in original_data.columns:
        original_data = original_data.reset_index()
        original_data.rename(columns={'index': 'index'}, inplace=True)
    
    return original_data

# 3. 유사한 와인의 추가 정보 추출 및 중복 제거
def get_wine_details(similar_wines_df, original_data):
    # 유사한 와인의 인덱스 리스트 추출
    similar_indices = similar_wines_df['index'].tolist()
    
    # 원본 데이터에서 해당 인덱스에 해당하는 와인 정보 추출
    wine_details = original_data[original_data['index'].isin(similar_indices)]
    
    # 유사한 와인 순서에 맞춰 정렬
    wine_details = wine_details.set_index('index').reindex(similar_indices).reset_index()
    
    # 거리 정보 추가
    wine_details = wine_details.merge(similar_wines_df, on='index')
    
    # 필요한 컬럼만 선택
    columns_of_interest = [
        'index', 'wine_name', 'wine_rating', 'wine_price',
        'wine_country', 'wine_region', 'wine_winery', 'wine_type', 'wine_grape', 'wine_image', 'distance'
    ]
    
    # 정보 추출
    wine_details = wine_details[columns_of_interest]
    
    # 와인 이름의 숫자 이전 부분만 추출하여 중복 제거
    def extract_name_prefix(name):
        match = re.match(r'^(.*\D)', name)
        return match.group(1) if match else name
    
    wine_details['name_prefix'] = wine_details['wine_name'].apply(extract_name_prefix)
    
    # 이름의 접두사로 중복 제거
    wine_details = wine_details.drop_duplicates(subset='name_prefix')
    
    # 원래의 최대 10개 와인으로 제한
    if len(wine_details) > 50:
        wine_details = wine_details.head(50)
    
    # 거리로 정렬
    wine_details = wine_details.sort_values(by='distance')
    
    # 불필요한 컬럼 삭제
    wine_details = wine_details.drop(columns='name_prefix')
    
    return wine_details

# `result_df`와 `data/Clean_Red_data.csv`에서 가장 유사한 와인 찾기
csv_file_path = 'data/Clean_Red_data.csv'
original_data = load_and_prepare_data(csv_file_path)

# `result_df`가 하나의 와인만 포함한다고 가정
target_index = 0  # `result_df`의 첫 번째 와인

# 가장 유사한 와인 찾기
similar_wines_indices = find_similar_wines_from_result_df(target_index, original_data)

# 유사한 와인의 상세 정보 추출
wine_details = get_wine_details(similar_wines_indices, original_data)

# 가격 필터링 적용 (예: '150000'으로 필터링)
price_range = '50000'
filtered_wine_details = filter_by_price(wine_details, price_range)

# 결과 출력
print(filtered_wine_details)


    index                                     wine_name  wine_rating  \
5      51                          Original Merlot 2004          3.5   
6    1168  Plantier de La Reine Bordeaux Superieur 2004          3.7   
7     674                                 L'Heravi 2010          3.5   
8     234               Cabernet Sauvignon Reserva 2005          3.5   
9     900                                Red Blend 2002          4.1   
11    568                       Cabernet Sauvignon 2013          3.2   
12    603                                Gotim Bru 2014          3.7   
15   1679                       Higueruela Crianza 2010          3.6   
16   1256                             Colors Tinto 2015          3.6   
18    168              Varietal Cabernet Sauvignon 2007          3.1   

    wine_price  wine_country         wine_region          wine_winery  \
5      20834.0        France           Pays d'Oc           JP. Chenet   
6      32502.0        France  Bordeaux Supérieur    Château d

<>:10: SyntaxWarning: invalid escape sequence '\$'
<>:10: SyntaxWarning: invalid escape sequence '\$'
C:\Users\KOSMO\AppData\Local\Temp\ipykernel_11984\3071113036.py:10: SyntaxWarning: invalid escape sequence '\$'
  df_red_wine['wine_price'] = df_red_wine['wine_price'].replace('[\$,]', '', regex=True).astype(float)
c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
